In [1]:
#Tensorflow library. Used to implement machine learning models
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
#import tensorflow as tf
#Numpy contains helpful functions for efficient mathematical calculations
import numpy as np
#Dataframe manipulation library
import pandas as pd
#Graph plotting library
import matplotlib.pyplot as plt
%matplotlib inline

2024-02-26 21:04:17.573894: W external/org_tensorflow/tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-02-26 21:04:18.906909: W external/org_tensorflow/tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-02-26 21:04:18.977188: W external/org_tensorflow/tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory


Instructions for updating:
non-resource variables are not supported in the long term


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
#!!!!path will have to change if moved to a different system!!!!
#Loading in the movies dataset
movies_df = pd.read_csv('/home/jack_shannon_student_uml_edu/RBM test/data/ml-1m/movies_fix.dat', sep = '::', header = None)
#Loading in the ratings dataset
ratings_df = pd.read_csv('/home/jack_shannon_student_uml_edu/RBM test/data/ml-1m/ratings.dat', sep = '::', header = None)
#Rename the columns in the dataset
movies_df.columns = ['MovieID', 'Title', 'Genres']
ratings_df.columns = ['UserID', 'MovieID', 'Rating', 'Timestamp']

/modules/apps/conda-environments/colabfold/lib/python3.7/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [3]:
len(ratings_df)

1000209

In [4]:
#Create a seperate indexing column as we cannot use MovieID
movies_df['List Index'] = movies_df.index
movies_df.head()

,MovieID,Title,Genres,List Index
0,1,Toy Story (1995),Animation|Children's|Comedy,0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,1
2,3,Grumpier Old Men (1995),Comedy|Romance,2
3,4,Waiting to Exhale (1995),Comedy|Drama,3
4,5,Father of the Bride Part II (1995),Comedy,4


In [5]:
#Merge movies with ratings by MovieID as foriegn key
merged_df=movies_df.merge(ratings_df, on='MovieID')

In [6]:
#Drop cols
merged_df = merged_df.drop('Timestamp', axis = 1).drop('Title', axis = 1).drop('Genres', axis = 1)

In [7]:
userGroup = merged_df.groupby('UserID')
print(len(userGroup))

6040


In [8]:
#Formatting the data for inout into the RBM. First we normalize the user ratings into a list - trX
#I hate the wording of this
#No. of users in training
UsedUsers = 1000
#create list
trX = []
#for each user in the group
for UserID, curUser in userGroup:
    #Temp variable that stores every movie's rating
    temp = [0]*len(movies_df)
    #For Each movie in the curUser's movie list
    for num, movie in curUser.iterrows():
        #Divide ratings by 5 and store it
        temp[movie['List Index']] = movie['Rating']/5.0
    
    #Now add the list of ratings into the training list
    trX.append(temp)
    #Check to see if we finished adding in the amount of users for training
    if UsedUsers == 0:
        break
    UsedUsers -= 1

In [10]:
# #Building an RBM with Tf. Determine no. of hidden layers and creating
# respective placeholder variables for storing visible and hidden layer biases
# and weights that connect the 2 layers.
hiddenUnits = 11 #Arbitrary
visibleUnits = len(movies_df)
vb = tf.placeholder("float", [visibleUnits]) #No. of unique movies
hb = tf.placeholder("float", [hiddenUnits]) #no. of features 
W = tf.placeholder("float", [visibleUnits, hiddenUnits])

In [65]:
#create visible and hidden layer units and set their activation function
# we use sigmoid and relu functions
#Input Preprocessing
v0 = tf.placeholder("float", [None, visibleUnits])
#v0 = tf.placeholder("float", [visibleUnits])
_h0 = tf.nn.sigmoid(tf.matmul(v0, W) + hb)
h0 = tf.nn.relu(tf.sign(_h0 - tf.random_uniform(tf.shape(_h0))))

#Reconstruction
_v1 = tf.nn.sigmoid(tf.matmul(h0, tf.transpose(W)) + vb)
v1 = tf.nn.relu(tf.sign(_v1 - tf.random_uniform(tf.shape(_v1))))
h1 = tf.nn.sigmoid(tf.matmul(v1, W) + hb)

In [63]:
#set rbm training parameters
print(v0)

Tensor("Placeholder_4:0", shape=(?, 3883), dtype=float32)


In [13]:
#learning rate
alpha = 0.1
#create gradients
w_pos_grad = tf.matmul(tf.transpose(v0), h0)
w_neg_grad = tf.matmul(tf.transpose(v1), h1)

#calculate contrastive divergence to maximize
CD = (w_pos_grad - w_neg_grad) / tf.to_float(tf.shape(v0)[0])

#create methods to update the weights and biases
update_w = W + alpha*CD
update_vb = vb + alpha*tf.reduce_mean(v0 - v1, 0)
update_hb = hb + alpha*tf.reduce_mean(h0 - h1, 0)

Instructions for updating:
Use `tf.cast` instead.


Instructions for updating:
Use `tf.cast` instead.


In [14]:
#set error function - MSE
err = v0 - v1
err_sum = tf.reduce_mean(err*err)

In [15]:
#initialze variables
#Current weight
cur_w = np.zeros([visibleUnits, hiddenUnits], np.float32)
#Current visible unit biases
cur_vb = np.zeros([visibleUnits], np.float32)
#Current hidden unit biases
cur_hb = np.zeros([hiddenUnits], np.float32)
#Previous weight
prv_w = np.zeros([visibleUnits, hiddenUnits], np.float32)
#Previous visible unit biases
prv_vb = np.zeros([visibleUnits], np.float32)
#Previous hidden unit biases
prv_hb = np.zeros([hiddenUnits], np.float32)
sess = tf.Session()
sess.run(tf.global_variables_initializer())


In [56]:
#set entopry function for whole system
def partition(cur_w,cur_vb,cur_hb,v1,h1):
    Hm = np.zeros([visibleUnits, hiddenUnits], np.float32)
    Z  = 0
    for i in range(0,visibleUnits):
        for j in range(0,hiddenUnits):
            Hm[i][j] = (-cur_w[i][j]*v1[i]*h1[j]-cur_vb[i]-cur_hb[j])
            Z += np.exp(cur_w[i][j]*v1[i]*h1[j]+cur_vb[i]+cur_hb[j])
    return [Hm,Z]
#returns probability of whole System with totlal Entropy!
def probEntropy(cur_w,cur_vb,cur_hb,v1,h1):
    tool = partition(cur_w,cur_vb,cur_hb,v1,h1)
    pT = np.zeros([visibleUnits, hiddenUnits], np.float32)
    Entropy = 0
    for i in range(0,visibleUnits):
        for j in range(0,hiddenUnits):
            pT[i][j]=(np.divide(np.exp(-tool[0]),tool[1]))
            Entropy+=-pT[i][j]*np.log(pT[i][j])
    return [pT,Entropy]

In [52]:
print(v1)

Tensor("Relu_1:0", shape=(?, 3883), dtype=float32)


In [67]:
epochs = 20
batchsize = 100
errors = []
entropy =[] #new addition to this system for analysis
for i in range(epochs):
    for start, end in zip( range(0, len(trX), batchsize), range(batchsize, len(trX), batchsize)):
        batch = trX[start:end]
        cur_w = sess.run(update_w, feed_dict={v0: batch, W: prv_w, vb: prv_vb, hb: prv_hb})
        cur_vb = sess.run(update_vb, feed_dict={v0: batch, W: prv_w, vb: prv_vb, hb: prv_hb})
        cur_nb = sess.run(update_hb, feed_dict={v0: batch, W: prv_w, vb: prv_vb, hb: prv_hb})
        prv_w = cur_w
        prv_vb = cur_vb
        prv_hb = cur_nb
    errors.append(sess.run(err_sum, feed_dict={v0: trX, W: cur_w, vb: cur_vb, hb: cur_nb}))
    visl = sess.run(v1,feed_dict={v0: trX, W: cur_w, vb: cur_vb, hb: cur_nb})
    hidl = sess.run(h1,feed_dict={v0: trX, W: cur_w, vb: cur_vb, hb: cur_nb})
    #entropy.append(sess.run(probEntropy(cur_w,cur_vb,cur_hb,v1,h1)[1],feed_dict={v1: visl, h1: hidl, v0: trX, W: cur_w, vb: cur_vb, hb: cur_nb}))
    print (errors[-1])
    print(visl1[1])
    #print (entropy[-1])
plt.plot(errors)
plt.ylabel('Entropy')
plt.xlabel('Epoch')
plt.show()

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_4' with dtype float and shape [?,3883]
	 [[node Placeholder_4
 (defined at /tmp/ipykernel_1836411/3217241297.py:4)
]]

Errors may have originated from an input operation.

Operation defined at: (most recent call last)
>>>   File "/modules/apps/conda-environments/colabfold/lib/python3.7/runpy.py", line 193, in _run_module_as_main
>>>     "__main__", mod_spec)
>>> 
>>>   File "/modules/apps/conda-environments/colabfold/lib/python3.7/runpy.py", line 85, in _run_code
>>>     exec(code, run_globals)
>>> 
>>>   File "/modules/apps/conda-environments/colabfold/lib/python3.7/site-packages/ipykernel_launcher.py", line 17, in <module>
>>>     app.launch_new_instance()
>>> 
>>>   File "/modules/apps/conda-environments/colabfold/lib/python3.7/site-packages/traitlets/config/application.py", line 992, in launch_instance
>>>     app.start()
>>> 
>>>   File "/modules/apps/conda-environments/colabfold/lib/python3.7/site-pac

In [ ]:
#Selecting the input user
inputUser = [trX[999]]

In [ ]:
#Feeding in the user and reconstructing the input
hh0 = tf.nn.sigmoid(tf.matmul(v0, W) + hb)
vv1 = tf.nn.sigmoid(tf.matmul(hh0, tf.transpose(W)) + vb)
feed = sess.run(hh0, feed_dict={ v0: inputUser, W: prv_w, hb: prv_hb})
rec = sess.run(vv1, feed_dict={ hh0: feed, W: prv_w, vb: prv_vb})

In [ ]:
scored_movies_df_86 = movies_df
scored_movies_df_86["Recommendation Score"] = rec[0]
scored_movies_df_86.sort_values(["Recommendation Score"], ascending=False)

In [ ]:
merged_df.iloc[999]

In [ ]:
scored_movies_df_86 = movies_df
scored_movies_df_86["Recommendation Score"] = rec[0]
scored_movies_df_86.sort_values(["Recommendation Score"], ascending=False)

In [ ]:
#Merging movies_df with ratings_df by MovieID
merged_df_86 = scored_movies_df_86.merge(movies_df_86, on='MovieID', how='outer')
#Dropping unecessary columns
merged_df_86 = merged_df_86.drop('List Index_y', axis=1).drop('UserID', axis=1)

In [ ]:
merged_df_86.sort_values(["Rating"], ascending=True).head(60)